In [1]:
%run preprocess_functions.ipynb
%run data_augmentation.ipynb

In [2]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import matplotlib.pyplot as plt
from torch import optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, precision_recall_curve, auc, roc_auc_score, roc_curve, multilabel_confusion_matrix

In [3]:
DIRECTORY_PATH = 'one_eye_images'
AUGMENTED_DIRECTORY_PATH ="original_and_augmented_images"

# model for one eye input

Dataset class

In [5]:
class CustomDataset(Dataset):
    def __init__(self, dataset_X, dataset_y, image_directory_path, transfrom=None, include_preprocess_function=False, mode='train'):
        self.X = dataset_X
        self.y = dataset_y
        self.image_directory_path = image_directory_path
        # self.grouped_by_id_data = self.dataset.groupby('patien_id') #groups in format of { patient_id: [indx1, indx2] }
        if mode == 'train':
            if include_preprocess_function:
                self.transform = transfrom or transforms.Compose([
                    # transforms.RandomHorizontalFlip(),
                    # transforms.RandomRotation(degrees=90),
                    transforms.Lambda(preprocess),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #values from image net, better for pretrained models, can be changed to dataset values
                ])
            else:
                self.transform = transfrom or transforms.Compose([
                    transforms.Resize((224, 224)),
                    # transforms.RandomHorizontalFlip(),
                    # transforms.RandomRotation(degrees=90),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #values from image net, better for pretrained models, can be changed to dataset values
                ])
        else:
            self.transform = transfrom or transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #values from image net, better for pretrained models, can be changed to dataset values
                ])
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        # eye_image = cv2.imread(os.path.join(self.image_directory_path, self.X.iloc[index]['image_id']))
        eye_image = Image.open(os.path.join(self.image_directory_path, self.X.iloc[index]['image_id']), 'r')
        diagnosis = self.y.iloc[index][['diabetic_retinopathy', 'amd', 'hypertensive_retinopathy', 
                                                'normal_eye', 'glaucoma', 'cataract']].to_numpy(dtype=np.float32)
        age = self.X.iloc[index]['patient_age']
        sex = self.X.iloc[index]['patient_sex']
        if self.transform:
           eye_image = self.transform(eye_image)
        
        data = {"eye_image": eye_image,
                "diagnosis": torch.tensor(diagnosis, dtype=torch.float32),
                "metadata": torch.tensor(np.array([age, sex]), dtype=torch.float32)}
        
        return data['eye_image'], data['metadata'], data['diagnosis']

In [6]:
# class VGG(nn.Module):
#     def __init__(
#         self, features: nn.Module, num_classes: int = 1000, init_weights: bool = True, dropout: float = 0.5
#     ) -> None:
#         super().__init__()
#         self.features = features
#         self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
#         self.classifier = nn.Sequential(
#             nn.Linear(512 * 7 * 7, 4096),
#             nn.ReLU(True),
#             nn.Dropout(p=dropout),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Dropout(p=dropout),
#             nn.Linear(4096, num_classes),
#         )
#         if init_weights:
#             for m in self.modules():
#                 if isinstance(m, nn.Conv2d):
#                     nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
#                     if m.bias is not None:
#                         nn.init.constant_(m.bias, 0)
#                 elif isinstance(m, nn.BatchNorm2d):
#                     nn.init.constant_(m.weight, 1)
#                     nn.init.constant_(m.bias, 0)
#                 elif isinstance(m, nn.Linear):
#                     nn.init.normal_(m.weight, 0, 0.01)
#                     nn.init.constant_(m.bias, 0)

#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         x = self.features(x)
#         x = self.avgpool(x)
#         x = torch.flatten(x, 1)
#         x = self.classifier(x)
#         return x


Model class

1. vgg 

In [26]:
class NetworkVGG(nn.Module):
    def __init__(self):
        super(NetworkVGG, self).__init__()
        
        self.eye_input = models.vgg16(weights='VGG16_Weights.DEFAULT').requires_grad_(False)
        self.eye_input = nn.Sequential(*list(self.eye_input.children())[:-1]) #removing the classifier layer 
        # for param in self.eye_input.parameters():
        #     param.requires_grad = False

        # self.eye_input = nn.Sequential(
            
        # )
            
        self.metadata_input = nn.Sequential(
            nn.Linear(2, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 64),
            nn.ReLU(inplace=True)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(25088 + 64, 4096), #4096
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 6),
            # nn.Softmax(dim=1)
        )
        
    def forward(self, eye_image, metadata): 
        vgg = self.eye_input(eye_image)
        vgg = torch.flatten(vgg, 1)
        meta = self.metadata_input(metadata)

        # display(vgg)
        # display(meta)
        concatenated = torch.cat((vgg, meta), dim=1)
        result = self.classifier(concatenated)
        return result

train function

In [6]:
def train(dataloader, model, loss_function, optimizer):
    size = len(dataloader.dataset)
    running_loss = 0.
    model.train()
    for batch, (image, metadata, diagnosis) in enumerate(dataloader): #(image, metadata, diagnosis)

        prediction = model(image, metadata) #, metadata
        # print("prediction", prediction[0])
        # print("diagnosis", diagnosis[0])
        loss = loss_function(prediction, diagnosis)
        running_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 2 == 0:
            loss, current = loss.item(), batch * len(diagnosis)
            print(f"loss: {loss:>7f}, [{current:>5d}/{size:>5d}]")
    return running_loss / len(dataloader)

In [7]:
def test_and_validation(dataloader, model, loss_function):
    val_loss = 0.
    model.eval()
    with torch.no_grad():
        for (image, metadata, diagnosis) in dataloader: #(image, metadata, diagnosis)
            prediction = model(image, metadata) #, metadata
            batch_val_loss = loss_function(prediction, diagnosis)
            val_loss += batch_val_loss
    return val_loss / len(dataloader)

In [44]:
def test(dataloader, model, loss_function):
    num_batches = len(dataloader)
    test_loss = 0.
    model.eval()
    with torch.no_grad():
        for (image, metadata, diagnosis) in dataloader:
            prediction = model(image, metadata)
            batch_test_loss = loss_function(prediction, diagnosis)
            test_loss += batch_test_loss.item()
            print(test_loss)
    return test_loss / num_batches

In [6]:
data = pd.read_csv("final_one_eye_dataset.csv")
classes = ['diabetic_retinopathy', 'amd', 'hypertensive_retinopathy', 'normal_eye', 'glaucoma', 'cataract']
X = data[['image_id', 'patient_age', 'patient_sex']]
y = data[classes]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)


augmentation

In [30]:
data_for_augmentation = X_train.join(y_train)
# target_samples = y_train.value_counts().max()
# augmented_df = augment_dataset(data_for_augmentation, target_samples, classes, DIRECTORY_PATH, AUGMENTED_DIRECTORY_PATH, transform=None)

In [31]:
augmented_df = pd.read_csv("augmented_df.csv")
augmented_df.drop(columns=['Unnamed: 0'], inplace=True)

In [32]:
augmented_df.sample(5)

,image_id,patient_age,patient_sex,diabetic_retinopathy,amd,hypertensive_retinopathy,normal_eye,glaucoma,cataract
367,4673_left_aug0.jpg,85.0,1,1,0,0,0,0,0
6417,2137_left_aug6.jpg,44.0,2,0,0,0,0,0,1
4289,1453_right_aug4.jpg,77.0,1,0,0,0,0,1,0
4968,1421_left_aug1.jpg,60.0,1,0,0,0,0,1,0
3508,img00742_aug8.jpg,79.0,1,0,0,1,0,0,0


In [33]:
original_and_augmented_df = pd.concat([data_for_augmentation, augmented_df], ignore_index=True)
X_train = original_and_augmented_df[['image_id', 'patient_age', 'patient_sex']]
y_train = original_and_augmented_df[classes]

datasets and dataloaders

In [34]:
age_scaler = StandardScaler()

X_train['patient_age'] = age_scaler.fit_transform(X_train[['patient_age']])
X_val['patient_age'] = age_scaler.transform(X_val[['patient_age']])
X_test['patient_age'] = age_scaler.transform(X_val[['patient_age']])

train_dataset = CustomDataset(X_train, y_train, AUGMENTED_DIRECTORY_PATH)
validation_dataset = CustomDataset(X_val, y_val, AUGMENTED_DIRECTORY_PATH, mode='test')
test_dataset = CustomDataset(X_test, y_test, AUGMENTED_DIRECTORY_PATH, mode='test')

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

C:\Users\polin\AppData\Local\Temp\ipykernel_14888\1755045109.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['patient_age'] = age_scaler.fit_transform(X_train[['patient_age']])


In [35]:
epochs_vgg16 = 15
learning_rate_vgg = 0.001
model_vgg = NetworkVGG()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_vgg.parameters(), lr=learning_rate_vgg)
train_losses = []
val_losses = []

for t in range(epochs_vgg16):
    print(f"Epoch {t+1}\n-------------------------------")
    epoch_train_loss = train(train_dataloader, model_vgg, loss_function, optimizer)
    epoch_val_loss =test_and_validation(validation_dataloader, model_vgg, loss_function)
    train_losses.append(epoch_train_loss)
    val_losses.append(epoch_val_loss)
print("Done!")

Epoch 1
-------------------------------
loss: 1.796266, [    0/11334]
loss: 5.413529, [  256/11334]
loss: 5.883486, [  512/11334]
loss: 1.818259, [  768/11334]
loss: 1.505136, [ 1024/11334]
loss: 1.398294, [ 1280/11334]
loss: 1.444097, [ 1536/11334]
loss: 1.471624, [ 1792/11334]
loss: 1.278474, [ 2048/11334]
loss: 1.296580, [ 2304/11334]
loss: 1.213410, [ 2560/11334]
loss: 1.295469, [ 2816/11334]
loss: 1.182201, [ 3072/11334]
loss: 1.217734, [ 3328/11334]
loss: 1.261012, [ 3584/11334]
loss: 1.222866, [ 3840/11334]
loss: 1.156577, [ 4096/11334]
loss: 1.241525, [ 4352/11334]
loss: 1.078676, [ 4608/11334]
loss: 1.294641, [ 4864/11334]
loss: 1.161287, [ 5120/11334]
loss: 1.211407, [ 5376/11334]
loss: 1.121953, [ 5632/11334]
loss: 1.079645, [ 5888/11334]
loss: 1.185516, [ 6144/11334]
loss: 1.012404, [ 6400/11334]
loss: 1.054030, [ 6656/11334]
loss: 1.086447, [ 6912/11334]
loss: 1.035527, [ 7168/11334]
loss: 1.157171, [ 7424/11334]
loss: 1.012563, [ 7680/11334]
loss: 0.952883, [ 7936/11334]


KeyboardInterrupt: 

In [36]:
torch.save(model_vgg.state_dict(), 'vgg_model.pth')

In [37]:
train_losses

[1.3695705653576369,
 0.7781428319684575,
 0.5923326380467147,
 0.4782548225327824,
 0.4305681502551175,
 0.3712513761573963,
 0.3401051274176394,
 0.3178074627779843,
 0.29567769186550313,
 0.27077615076906225,
 0.2396045196592138]

In [38]:
val_losses

[tensor(1.1102),
 tensor(1.0592),
 tensor(1.0453),
 tensor(1.0799),
 tensor(1.0433),
 tensor(1.2642),
 tensor(1.0555),
 tensor(1.1894),
 tensor(1.3154),
 tensor(1.2449),
 tensor(1.2217)]

In [50]:
import pickle
with open('train_losses.pkl', 'wb') as file:
    pickle.dump(train_losses, file)
with open('val_losses.pkl', 'wb') as file:
    pickle.dump(val_losses, file)
with open('scaler.pkl', 'wb') as file:
    pickle.dump(age_scaler, file)
    
torch.save(train_dataloader, 'train_dataloader.pth')
torch.save(validation_dataloader, 'validation_dataloader.pth')
torch.save(test_dataloader, 'test_dataloader.pth')
torch.save(optimizer.state_dict(), 'optimizer.pth')